# Quantitative Trading Research Notebook

This notebook demonstrates how to use QuantLib for strategy research and backtesting.

## Setup

Make sure QuantLib is installed and your database is configured.

In [ ]:
# Import QuantLib research helpers
from quantlib.research import (
    load_symbol,
    run_backtest,
    plot_equity_curve,
    plot_trades,
    plot_drawdown,
    get_metrics_summary
)

# Import strategy classes
from quantlib.strategies.examples import MovingAverageStrategy

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Jupyter display settings
%matplotlib inline
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

## Load Data

Load market data for the symbol you want to analyze.

In [ ]:
# Load data for a symbol
symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2023-01-01'

data = load_symbol(symbol, start_date, end_date)
print(f"Loaded {len(data)} rows of data for {symbol}")
data.head()

## Define and Run Strategy

Create a strategy and run a backtest.

In [ ]:
# Create strategy with parameters
strategy = MovingAverageStrategy({
    'short_window': 20,
    'long_window': 50
})

# Run backtest
results = run_backtest(
    strategy=strategy,
    data=data,
    symbol=symbol,
    initial_capital=100000,
    commission=1.0,
    slippage=0.0
)

print("Backtest completed!")

## View Metrics

Display performance metrics in a formatted table.

In [ ]:
# Get metrics summary
metrics_summary = get_metrics_summary(results['metrics'])
display(metrics_summary)

## Visualize Results

Plot equity curve, trades, and drawdown.

In [ ]:
# Plot equity curve
plot_equity_curve(results['equity_curve'], title=f"{symbol} Equity Curve")

In [ ]:
# Plot trades on price chart
plot_trades(data, results['trades'], results['equity_curve'], title=f"{symbol} Trading Signals")

In [ ]:
# Plot drawdown
plot_drawdown(results['equity_curve'], title=f"{symbol} Drawdown")

## Analyze Trades

Examine individual trades and trade statistics.

In [ ]:
# View trades
trades = results['trades']
print(f"Total trades: {len(trades)}")
if not trades.empty:
    display(trades.head(10))

## Parameter Optimization Example

Test different parameter combinations to find optimal settings.

In [ ]:
# Test different parameter combinations
short_windows = [10, 20, 30]
long_windows = [50, 100, 150]

optimization_results = []

for short in short_windows:
    for long in long_windows:
        if short >= long:
            continue  # Skip invalid combinations
        
        strategy = MovingAverageStrategy({
            'short_window': short,
            'long_window': long
        })
        
        result = run_backtest(strategy, data, symbol)
        
        optimization_results.append({
            'short_window': short,
            'long_window': long,
            'sharpe_ratio': result['metrics'].get('sharpe_ratio', 0),
            'total_return': result['metrics'].get('total_return', 0),
            'max_drawdown_pct': result['metrics'].get('max_drawdown_pct', 0)
        })

# Create DataFrame and sort by Sharpe ratio
opt_df = pd.DataFrame(optimization_results)
opt_df = opt_df.sort_values('sharpe_ratio', ascending=False)
display(opt_df)

## Further Analysis

You can perform additional analysis on the results:
- Compare multiple strategies
- Analyze returns distribution
- Calculate rolling metrics
- Perform statistical tests

The results dictionary contains:
- `results['equity_curve']`: Equity over time
- `results['trades']`: Trade history DataFrame
- `results['returns']`: Returns Series
- `results['metrics']`: Performance metrics dict
- `results['results']`: Full backtest results dict